In [9]:
import re

In [2]:
with open('data/input_24.txt') as fh:
    file_input = fh.read().strip()

In [19]:
file_input.split('\n')

['Immune System:',
 '123 units each with 8524 hit points with an attack that does 612 slashing damage at initiative 11',
 '148 units each with 4377 hit points (weak to slashing, bludgeoning) with an attack that does 263 cold damage at initiative 1',
 '6488 units each with 2522 hit points (weak to fire) with an attack that does 3 bludgeoning damage at initiative 19',
 '821 units each with 8034 hit points (immune to cold, bludgeoning) with an attack that does 92 cold damage at initiative 17',
 '1163 units each with 4739 hit points (weak to cold) with an attack that does 40 bludgeoning damage at initiative 14',
 '1141 units each with 4570 hit points (weak to fire, slashing) with an attack that does 32 radiation damage at initiative 18',
 '108 units each with 2954 hit points with an attack that does 262 radiation damage at initiative 8',
 '4752 units each with 6337 hit points (weak to bludgeoning, cold; immune to slashing) with an attack that does 13 cold damage at initiative 20',
 '4489 u

In [20]:
test_input = """Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4"""

In [227]:
class Group(object):
    def __init__(self, units=0, hp=0, ad=0, boost=0, attack_type=None, initiative=0, weaknesses=[], immunities=[], team=0):
        self.units = units
        self.hp = hp
        self.ad = ad + boost
        self.attack_type = attack_type
        self.initiative = initiative
        self.weaknesses = weaknesses
        self.immunities = immunities
        self.team = team
        
    def get_effective_power(self):
        return self.units * self.ad
    
    def target_selection_order(self):
        return (self.get_effective_power(), self.initiative)
    
    def target_damage(self, target):
        if self.attack_type in target.immunities:
            return 0
        damage = self.units * self.ad
        if self.attack_type in target.weaknesses:
            damage *= 2
        return damage
    
    def deal_damage(self, damage):
        self.units -= damage // self.hp
    
    def __repr__(self):
        return ">units: {} | ad: {} | ep: {} | hp: {} | ini: {} | w: {} | i: {} | at: {} | team: {}<\n".format(self.units, self.ad, self.get_effective_power(), self.hp, self.initiative, self.weaknesses, self.immunities, self.attack_type, self.team)

class Battle(object):
    def __init__(self, groups=[], maxfights=float('inf')):
        self.groups = groups
        self.targets = []
        self.maxfights = maxfights
        
    def cont(self):
        n0 = len([g for g in self.groups if g.team == 0])
        n1 = len([g for g in self.groups if g.team == 1])
        if n0 > 0 and n1 > 0:
            return True
        return False
        
    def attack_order(self):
        initiatives = [g.initiative for g in self.groups]
        return sorted(range(len(initiatives)), key=initiatives.__getitem__, reverse=True)
        
    def target_selection(self):
        self.targets = []
        self.groups = sorted(self.groups, key=lambda x: x.target_selection_order(), reverse=True)
        for g in self.groups:
            targets = sorted([(g.target_damage(t), t.get_effective_power(), t.initiative, t) for t in self.groups if t.team != g.team and t not in self.targets], reverse=True)
            if targets and targets[0][0] > 0:
                self.targets.append(targets[0][3])
            else:
                self.targets.append(None)
#         print(self.groups) 
#         print(self.targets)

    def attack_phase(self):
        for i in self.attack_order():
            group = self.groups[i]
            target = self.targets[i]
            if group.units <= 0 or target is None:
                continue
            damage = group.target_damage(target)
            target.deal_damage(damage)
#             print(group, target, damage)
        self.groups = [g for g in self.groups if g.units > 0]
    
    def fight(self):
        self.i = 0
        while self.cont() and self.i < self.maxfights:
#             print("# {}".format(i))
            self.target_selection()
            self.attack_phase()
            self.i += 1
        

In [216]:
def parse_input(inp):
    team = 0
    for line in inp.split('\n'):
        if line == "Infection:":
            team = 1
        stats = map(int, re.findall('\d+', line))
        if len(stats) != 4:
            continue
        weaknesses = re.findall('weak to ([^;)]*)', line)
        if weaknesses:
            weaknesses = weaknesses[0].split(', ')
        immunities = re.findall('immune to ([^;)]*)', line)
        if immunities:
            immunities = immunities[0].split(', ')    
#         print(re.findall('([a-z]*) damage', line))
        attack_type = re.findall('([a-z]*) damage', line)[0]
        
        units, hp, ad, initiative = stats
        yield Group(units=units, hp=hp, ad=ad, attack_type=attack_type, initiative=initiative, weaknesses=weaknesses, immunities=immunities, team=team)

Test A

In [217]:
groups = list(parse_input(test_input))
battle = Battle(groups=groups)

battle.fight()

sum([g.units for g in battle.groups])

5216

Part A

In [218]:
groups = list(parse_input(file_input))
battle = Battle(groups=groups)

battle.fight()

sum([g.units for g in battle.groups])

21743

Test B

In [219]:
boost = 1570
groups = list(parse_input(test_input))
for g in groups:
    if g.team == 0:
        g.ad += boost
battle = Battle(groups=groups)

battle.fight()

sum([g.units for g in battle.groups])

51

Part B

In [249]:
boost = 61 # 55..61
groups = list(parse_input(file_input))

for g in groups:
    if g.team == 0:
        g.ad += boost
battle = Battle(groups=groups, maxfights=10000)

battle.fight()

battle.groups[0].team

0

In [253]:
sum([g.units for g in battle.groups])

884

In [251]:
battle.targets

[>units: -8 | ad: 1 | ep: -8 | hp: 6367 | ini: 5 | w: ['fire'] | i: ['radiation'] | at: slashing | team: 1<,
 >units: 884 | ad: 81 | ep: 71604 | hp: 9894 | ini: 12 | w: ['slashing'] | i: [] | at: slashing | team: 0<]

In [252]:
battle.groups[1].target_damage(battle.groups[0])

IndexError: list index out of range

In [201]:
groups = list(parse_input(file_input))